# LMB Python DNN Course Part 2

### Importing modules

The first commands in this notebook perform some initial set-up and module imports, as we did in Part 1. 

In [ ]:
%matplotlib inline
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np  # Load the NumPy module, assign it the name "np" for convienence
from matplotlib import pyplot as plt
from matplotlib import cm

plt.rcParams["figure.figsize"] = (16,8) # Set plot size

from keras import models, layers, regularizers
from keras.datasets import mnist
from keras.utils import to_categorical

We will also use the same function from Part 1 to plot the history of the DNN training, i.e. to report the loss and accuracy, for both training and test data, during the training iterations.

In [ ]:
def plot_training_history(*histories):
    cmap = cm.get_cmap('tab10')
    fig, (ax1, ax2) = plt.subplots(1, 2)
    plot_options = {'linewidth':2, 'alpha':0.5} # A dictionary of inputs for all charts
    
    for i, history in enumerate(histories):
        hd = history.history
        n = np.arange(len(hd['loss'])) + 1
        plot_options['color'] = cmap(float(i % 10)/10)
        
        ax1.plot(n, hd['loss'], label='Train %d' % i,
                 linestyle='--', **plot_options)
        ax1.plot(n, hd['val_loss'], label='Test %d' % i,
                 **plot_options)
        ax1.set_title('Loss')
        ax1.set_xlabel('Iteration')
        
        ax2.plot(n, hd['accuracy'], label='Train %d' % i,
                 linestyle='--', **plot_options)
        ax2.plot(n, hd['val_accuracy'], label='Test %d' % i,
                 **plot_options)
        ax2.set_title('Accuracy')
        ax2.set_xlabel('Iteration')
        
    ax1.legend()    
    ax2.legend()
    plt.show()

## Predicting ptotein transmembrane spans

We now turn to our second example dataset which relates to biological sequences. The objective here will be to train a DNN to predict the location and extent of a protein's transmembrane span(s), if it has any.  

For this we will use an existing, published dataset that is provided by the paper that goes with the [TMHMM](https://www.ncbi.nlm.nih.gov/pubmed/11152613) transmembrane prediction program. As the name suggests this method used a hidden markov model (HMM) to learn and perform predictions. We hope to do better using a DNN.

In some sense, the TM spans of proteins are relatively easy features to predict from protein sequence alone because the TM span sits in a hydrophobic lipid bilayer and so the amino acids will tend to be substantially hydrophobic in nature. However, the situation is not so straightforward when charged residues are present in the TM span (can be a false negative), when aqueous globular domains have long internal hydrophobic helices (can be false positive), when multi-span TM proteins have small internal helicesm, and when the protein possesses a hydrophobic signalling peptide (e.g. to gain entry to the secretory system), which then is cleaved from the final protein.

<img src="images_ML/tm_prot.png" style="float:left;width:200px;">

The input to our DNNs will be amino acid sequences, albeit in an encoded form. And the output will be a prediction of 'membrane' or 'non-membrane' categories. Just like the original TMHMM program we will also further separate the non-membrane residues into an internal/cytoplasmic class and an external/lumenal/exoplamic class, and thus also give a prediction of the protein's membrane spanning topology.

### Training data

The training and test data will come from files that were provided when the TMHMM program was published. This has saved a lot of effort for using this as a tutorial dataset and we can trust that it is well curated. 

Because the training and test data are coming from bespoke formatted text files we first construct a function that can read the data into the NumPy arrays that we want to work with. The format of the file for each protein entry looks like the below example, i.e. with a protein amino acid sequence and corresponding codes to indicate whether a residue is inside `i`, membraneous `M`, or outside`o`:

In [ ]:
with open('set160.labels') as file_obj:
    for i in range(4):
        print(file_obj.readline().strip())

The function to read these data is a little fiddly, but a Python function that does this is provided below. 
A second training set was release with a later version of the TMHMM method; this has 243 proteins in it compared to the original 160. We will consider both in this tutorial.

### Variable size input

One of the innate problems when dealing with many types of biological sequence is that the lengths of the sequences can vary. This is an issue for neural networks that operate with a fixed size of inputs.

The way that we will deal with this here is to consider a sliding window, which will be of a fixed size. In other words we will only be looking at part of the protein sequence at one time. For the purposes of predicting TM spans this approach will work as long as the window is wide enough. 

We will be predicting the inside `"i"`, outside `"o"` or membrane `"M"` category of the window's central residue(s), and so the extent of the window provides a sequence context either side of the middle. Naturally this context needs to be large enough to provide sufficient information for the prediction. For example if the window were only a few residues then this would not see the whole of a TM span and there would be no way to distinguish between small hydrophobic segments in non-membrane situations. We find that more than 50 residues works fairly well, as this is wide enough to cover a large TM span and a few residues either side; included some non-membrane residues is important for assessing the inside/outside topology; the topology flips either side of the TM span.

When using a sliding window the edges of the protein sequence are potentially a problem; we need to extend the window over the edge of the protein sequence so the middle residue (in the window) can be one of the ofrst or last amino acids. We will fox this issue by extending the sequence with padding (`'-'` characters) so that the window cannot fall of the edges. Accompanying this we introduce a new data label (also `'-'`), which in effect is a class for empty, edge space. This presents little problem for the DNN and indeed having an edge context with its own special label can be helpful if the rules differ near the N- and C-terminii. 

<img src="images_ML/TM_DNN_window.png" style="float:left;">

We will be taking a short cut with this example as the DNNs input for non-membrane residues will be the non-membrane parts of the membrane proteins, rather than fully aqueous proteins. Choosing the later would be a better option in reality, although you have to be careful to have a degree of balance between positive (TM) and negative (no TM) training data.

### Data augmentation

The practice of using a sliding sequence window means that each protein will be used many times as different input; i.e. with different offsets. We  could have looked at only sequence regions that are centred on each TM span. However, given that the number of TM protein examples is fairly small this would be somewhat limiting. Using a sliding window so that each sequence is used many times is effectively a technique called *data augmentation*; the same input is used many times with different offests or transformations (for image data the generally means scaling and rotation). Each different view of the data will present a different, albeit relates, context and effectively increases the amount of training data.

## Loading training and test data

The below function reads the TM data file and converts the 1-letter codes for the amino acid sequence and the transmembrane category into Python integer numbers. This is done with a sliding window of specified size (via `win_size`) such that there will be several offset inputs fpr each protein. The `is_main` variable determines if the protein, with all its sliding-window sub-sequences, goes into the main training data or else the test data. The `test_frac` determines what proportion of the data is used as test. Note that this is only an approximate fraction as the function will not split a protein between test and training; overlapping sub-sequences are highly correlated and would introduce bias in the training set. 


In [ ]:
from random import random, seed

CLASSES = 'ioM-'
AA = 'ACDEFGHIKLMNPQRSTVWY-'
AA_IDX = dict([(x,i) for i,x in enumerate(AA)])
LABEL_IDX  = dict([(x,i) for i,x in enumerate(CLASSES)])
NAA = len(AA)

def load_tm_data(file_path, win_size=25, pred_width=1,
                 stride=1, test_frac=0.1):

  half_width = int(win_size/2)
  pred_start = int(pred_width/2)
  pred_end = pred_width-pred_start

  train_data = []
  test_data = []
  train_labels = []
  test_labels = []
  
  prot_data = {}
    
  pad = CLASSES[-1] * half_width 
  with open(file_path) as file_obj:
    name = file_obj.readline().strip()[1:]
    
    while name:
      is_main = len(test_data) > (test_frac * len(train_data))
    
      seq = file_obj.readline()[2:-1]
      labels = file_obj.readline()[2:-1]
      null = file_obj.readline()

      n = len(seq)
      prot_data[name] = (seq, labels, is_main)     

      seq = pad + seq + pad
      labels = pad + labels + pad
        
      for i in range(half_width, n+half_width, stride):
        sub_seq = seq[i-half_width:i+half_width+1]
        sub_seq = [AA_IDX[aa] for aa in sub_seq]
        label = [labels[j] for j in range(i-pred_start,i+pred_end)] 
        label = [LABEL_IDX[j] for j in label]
        
        if is_main:
          train_data.append(sub_seq)
          train_labels.append(label)
        else:
          test_data.append(sub_seq)
          test_labels.append(label)

      name = file_obj.readline().strip()[1:]
 
  train_data = np.array(train_data)
  test_data = np.array(test_data)
  train_labels = np.array(train_labels)
  test_labels = np.array(test_labels)
  
  msg = 'Counts: training {:,}, test {:,} - Total proteins: {:,}'
  print(msg.format(len(train_labels), len(test_labels), len(prot_data)))
  
  return (train_data, train_labels), (test_data, test_labels), prot_data

### Properly separating the test data

When we split the test data from the main training data we must be very careful that the datasets are truly separate. For example we may want to guarantee that the members of an homologous protein family are always in the same segment. With our sliding window approach we can investigate the effect of not separating the data properly by spreading different sliding windows for the same proteins across both test and training data. We could achieve this simply by shuffling the data loaded from file so that it no longer separated into different protein origins.

### Preparing training data

We will start with a sliding window/region size of 20 residues. This will work to some degree, but we can easily investigate what happens with smaller or larger windows. With smaller regions we naturally get less sequence context, but training will be easier, as the input is smaller. Also, we will start with predicting only one, middle residue in our region and use the smaller of the TMHMM data files.

In [ ]:
win_size = 20
train, test, prot_data = load_tm_data('set160.labels', win_size) # set243.labels or set160.labels

The output from the data reading function is then split into input and output sections for both the main training data and the test data.

If we display the shapes of the resulting input data arrays we can see that the size of the last axis corresponds to the size of the sliding window. Within this the different amino acids (plus edge/gap) will be encoded, at least initially, as numbers 0-20. At this stage the outputs are simple, single categorical numbers 0-3 (for `'i'`, `'o'`, `'M'` or `'-'`).

In [ ]:
train_data, train_labels = train
test_data, test_labels = test

print(train_data.shape)
print(train_labels.shape)
print(train_data[0])
print(train_labels[0])

The output, categorical labels will be converted into a binary (one-hot) encoding; a 1.0 in a particular column sets the category and other values are 0.0.

In [ ]:
train_labels = to_categorical(train_labels, num_classes=4)
test_labels = to_categorical(test_labels, num_classes=4)
print(train_labels.shape)
print(train_labels[0])

Next we perform a similar procedure to prepare the input data, i.e. converting it to categorical binary matrices and then flattening into vectors. Here we explicitly state the number of categorical classes (`NAA`) because there is a small chance that a protein won't contain all the different types of amino acid.

In [ ]:
train_data = to_categorical(train_data, num_classes=NAA)
test_data = to_categorical(test_data, num_classes=NAA)

print(train_data.shape)
print(test_data.shape)

Because each data item is effectively a matrix, we flatten the last two axes into a single vector, as the DNN expects. The size of this vector is the inital matrix width time height.

In [ ]:
n, a, b = train_data.shape
m, a, b = test_data.shape

train_data = train_data.reshape(n, a*b)
test_data = test_data.reshape(m, a*b)

print(train_data.shape)
print(test_data.shape)

To make the loading of datasets easier later on. The above preparation for the input and output data is placed in a handy function called `prepare_data()`, which runs `load_tm_data()` internally and then performs the required categorical transformations etc. We can then run this function again when we switch to a different data file or change window size etc. 

This function also has some extra features which will be useful when trying more complex DNNs. Specifically there is: reshaping of the output labels in the section after `if train_labels.ndim > 2:`, for when we move from predicting just a single residues class to predicting classes for several residues; and an option to avoid converting the input data to a one-hot encoding (using `to_cat=False`) when we move to a learned encoding in the DNN itself.

At the end of the function we send back the prepared input and outputs for the main training and test data, the sizes of the input and output vectors as we will need this when we construct the DNNs and finally sequence information relating to each protein; so that we can easily go back to the original sequences

In [ ]:
def prepare_data(data_file, win_size=10, pred_width=1, stride=1, test_frac=0.1, to_cat=True):
  
  test, train, prot_data = load_tm_data(data_file, win_size, pred_width, stride, test_frac)
  train_data, train_labels = test
  test_data, test_labels = train

  train_labels = to_categorical(train_labels, num_classes=len(CLASSES))
  test_labels = to_categorical(test_labels, num_classes=len(CLASSES))

  if train_labels.ndim > 2:
    n, a, b = train_labels.shape
    m, a, b = test_labels.shape # Last two axes are the same size
    out_size = a * b

    train_labels = train_labels.reshape(n, out_size) # Flatten last two axes
    test_labels = test_labels.reshape(m, out_size)
  else:
    out_size = train_labels.shape[1]
  
  if to_cat:
    train_data = to_categorical(train_data, num_classes=len(AA))
    test_data = to_categorical(test_data, num_classes=len(AA))
    n, a, b = train_data.shape
    m, a, b = test_data.shape
    in_size = a * b
    train_data = train_data.reshape(n, in_size)
    test_data = test_data.reshape(m, in_size)
    
  else:
    in_size = train_data.shape[-1]
  
  return (train_data, train_labels), (test_data, test_labels), in_size, out_size, prot_data

We will re-load the training and test data to check this function.

In [ ]:
win_size = 40
train, test, in_size, out_size, prot_data = prepare_data('set160.labels', win_size)
(train_data, train_labels) = train
(test_data, test_labels) = test

Our initial (fully connected) DNN architecture follows the same lines that were used at the end of the numeral recogntition example presented earlier. The neural network model is a sequential one with a number of `Dense` and `Dropout` layers, where the width of each layer diminishes toward the output. However, we use a `for` loop to avoid repetitive code.
We set the last layer to (somewhat arbitrarily) be double the size of the output, so that it is definitely larger than the output vector, given the later will be changing in size as we tweak things.

In [ ]:
input_shape = [in_size]
nn1 = models.Sequential()

for width in [8*out_size, 2*out_size]:
    nn1.add(layers.Dense(width, activation='relu', input_shape=input_shape)) 
    input_shape = []

The last layer is set to be the same size as the output vectors and then the DNN is finalised. Note that the loss function and paired last-layer activation function are appropriate for categorical selection where only one out[ut value can be true.

In [ ]:
nn1.add(layers.Dense(out_size, activation='softmax'))
nn1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',])

Training is initially for 10 iteractions (`epoch=`), updating the DNN with batches of 512 training samples.

In [ ]:
history1 = nn1.fit(train_data, train_labels, epochs=10, batch_size=512,
                  validation_data=(test_data, test_labels))

The history of the loss and accuracy are plotted, as before, with our helper function `plot_training_history()`. We can see that predictor works  to an accuracy of about 75%. Note that this is the *per-residue* accuracy for the TM class assignment; even if some residues are incorrectly assigned the overall presence of a TM span will be more accurate. 

Unfortunately this network is subject to a notable amount of over-training; the loss and accuracy improve for the training data but not the test data. This is not unexpected, given that there are twenty different possible amino acids for each sequence position and so the number of total possible sequences is vast, and yet we have only sequenced a tiny fraction of this in the training data. In essence, long protein sequences are sparsely represented and thus potentially difficult to generalise.

In [ ]:
plot_training_history(history1)

To visualise the predictive output categories some more helper functions are provided: given a DNN model and a sequence `get_seq_scores()` will get the predicted scores for each output category using a sliding window and `plot_seq_predict()` will display this as graphs. Note that the neural network model object `nn` is input and the `.predict()` function is used after each sub-sequence has been correctly converted to the same vector form used in the training. This function plots the scores for the four possibilities (inside, outside, membrane or sequence edge) as line graphs along the length of a test protein sequence.

In [ ]:
def get_seq_scores(nn, seq, win_size, in_size, pred_size=1, categorical=True):
  n = len(seq)
  hw = int(win_size/2)
  pad = '-' * hw
  pseq = pad + seq + pad
  scores = np.zeros((n, 4))  
    
  for k in range(n):
    q = np.array([[AA_IDX[aa] for aa in pseq[k:k+hw+hw+1]]])
    if categorical:
      q = to_categorical(q, num_classes=NAA).reshape(1, in_size)
    else:
      q = q.reshape(1, in_size)
    
    out = nn.predict(q)
    
    if pred_size > 1:
      out = out.reshape(pred_size, 4)[int(pred_size/2)]
    
    scores[k] = out

  return scores

def plot_seq_predict(nn, seq, win_size, in_size, pred_size=1, categorical=True):
  scores = get_seq_scores(nn, seq, win_size, in_size, pred_size, categorical)

  opts = {'alpha':0.4, 'linewidth':3}
  i, o, m, x = scores.T

  fig, ax = plt.subplots()
  fig.set_size_inches(16.0, 3.0)  

  ax.plot(i, color='#B0B040', label='Inside', **opts)
  ax.plot(o, color='#4080FF', label='Outside', **opts)
  ax.plot(m, color='#B00000', label='Membrane', **opts)
  ax.plot(x, color='#808080', label='Edge', linestyle='--', **opts)
  ax.set_xlabel('Sequence position')
  ax.legend()
  plt.show()

The sequence predictions can now be run on any one-letter amino acid sequence.

In [ ]:
seq1 = "MYGKIIFVLLLSEIVSISASSTTGVAMHTSTSSSVTKSYISSQTNDTHKRDTYAATPRAHEVSEISVRTVYPPEEETGERVQLAHHFSEPEITLIIFGVMAGVIGTILLISYGIRRLIKKSPSDVKPLPSPDTDVPLSSVEIENPETSDQ"
seq2 = 'MWSTRSPNSTAWPLSLEPDPGMASASTTMHTTTIAEPDPGMSGWPDGRMETSTPTIMDIVVIAGVIAAVAIVLVSLLFVMLRYMYRHKGTYHTNEAKGTEFAESADAALQGDPALQDAGDSSRKEYFI'
plot_seq_predict(nn1, seq1, win_size, in_size)
plot_seq_predict(nn1, seq2, win_size, in_size)

Next we can investigate what happens if we change the widths of the full-connected layers and the number of layers. As you night expect, smaller networks suffer less from overtraining but having a network that is too shallow and/or narrow will reduce test accuracy. Conversely larger networks tend to over-train, get slow and may show no benefit to test accuracy.

## <font color=purple>Exercise 1: Tweaking layers</font>

<font color=purple>See if it is possible to optimise the widths and number of `Dense` layers to improve the DNN test data loss and accuracy.
</font>

In [ ]:
# Exercise code

input_shape = [in_size]
nn2 = models.Sequential()

# Insert more layers here....
    
nn2.add(layers.Dense(out_size, activation='softmax'))
nn2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',])

history2 = nn2.fit(train_data, train_labels, epochs=10, batch_size=512,
                  validation_data=(test_data, test_labels))

In [ ]:
plot_training_history(history1, history2)

### Input indow size

As well as the DNN parameters we can naturally also optimise the way in which the sequence is used as input.  For example here we train and test the network on input with a somewat larger window size:

In [ ]:
win_size = 75
train, test, in_size, out_size, prot_data = prepare_data('set160.labels', win_size)
(train_data, train_labels) = train
(test_data, test_labels) = test

In [ ]:
input_shape = [in_size]
nn3 = models.Sequential()

for width in [8*out_size, 2*out_size]:
    nn3.add(layers.Dense(width, activation='relu', input_shape=input_shape)) 
    input_shape = []
    
nn3.add(layers.Dense(out_size, activation='softmax'))
nn3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',])

history3 = nn3.fit(train_data, train_labels, epochs=10, batch_size=512,
                   validation_data=(test_data, test_labels))

Increasing the window size can give some improvement in accuracy, but larger input naturally makes the data more sparse and thus easier to overtrain.

In [ ]:
plot_training_history(history1, history3)

In [ ]:
plot_seq_predict(nn3, seq1, win_size, in_size)
plot_seq_predict(nn3, seq2, win_size, in_size)

## Parameter searches

Rather then manually searching for optimised parameters, these can naturally be sought in an automated manner with programming. A simple, though time consiming, approach here is an exhaustive seach that re-runs the DNN training many times and records the performance for a range (or grid) of paramaters. 

In [ ]:
accs = []
win_sizes = range(50, 100, 10)
for win_size in win_sizes:
    train, test, in_size, out_size, prot_data = prepare_data('set160.labels', win_size)
    (train_data, train_labels) = train
    (test_data, test_labels) = test
    input_shape = [in_size]
    nn = models.Sequential()

    for width in [8*out_size, 2*out_size]:
      nn.add(layers.Dense(width, activation='relu', input_shape=input_shape)) 
      input_shape = []
    
    nn.add(layers.Dense(out_size, activation='softmax'))
    nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',])

    h = nn.fit(train_data, train_labels, epochs=3, batch_size=512,
                validation_data=(test_data, test_labels))
    accs.append(h.history['val_accuracy'][-1])

In [ ]:
plt.plot(win_sizes, accs)
plt.show()

## Improving predictions by predicting more

When we inspect some of the TM category predictions for individual residue we can see that a common kind of error is a small segment (often only a single residue) that flips to the opposite inside/outside category compared to its neighbours. Naturally, this is not observed in the training class labels. Predicting the inside/outside class is more difficult that the membrane class, so mistakes are not surprising (generally this is base on something like the postive-inside rule near the TM span). This problem can be largely ameliorated by actually making more residue predictions. For example, rather than just predicting the i/o/M class for one central residue we can predict the class for several central residues. The DNN will learn that the categories are largely continuous and that transitions like 'iiooM' are never seen in the training data. In effect the adjacent residues give each other more context, albeit in the form of a predicted class.

In [ ]:
win_size = 60
pred_width = 3
train, test, in_size, out_size, prot_data = prepare_data('set160.labels', win_size, pred_width)
(train_data, train_labels) = train
(test_data, test_labels) = test

Note that when we train a network for an output where multiple classifications can be true at once we should swith to the `binary_crossentropy` loss function and pair this with the `sigmoid` activation function.

In [ ]:
input_shape = [in_size]
nn4 = models.Sequential()

for width in [8*out_size, 2*out_size]:
    nn4.add(layers.Dense(width, activation='relu', input_shape=input_shape)) 
    input_shape = []
    
nn4.add(layers.Dense(out_size, activation='sigmoid'))
nn4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',])

history4 = nn4.fit(train_data, train_labels, epochs=10, batch_size=512,
                   validation_data=(test_data, test_labels))

Hopefully you will see that predicting for multiple adjascent residues works really well with both better loss and accuracy.

In [ ]:
plot_training_history(history1, history4)

Visualising the score profiles along a couple of test sequences shows that the predictions are now smoother.

In [ ]:
plot_seq_predict(nn4, seq1, win_size, in_size, pred_width)
plot_seq_predict(nn4, seq2, win_size, in_size, pred_width)

## <font color=purple>Exercise 2: Choosing the loss function</font>

<font color=purple>Test what happens to the performance of the DNN if you keep using the `categorical_crossentropy` loss function and the corresponding `softmax` activation function for the last layer, as we did initially when only predicting for one residue.
</font>

In [ ]:
input_shape = [in_size]
nn3b = models.Sequential()

for width in [8*out_size, 2*out_size]:
    nn3b.add(layers.Dense(width, activation='relu', input_shape=input_shape)) 
    input_shape = []
    
# Fill in here

history3b = nn3b.fit(train_data, train_labels, epochs=10, batch_size=512,
                   validation_data=(test_data, test_labels))

plot_training_history(history3, history3b)

### Reducing over-training

We can employ the same regularisation and droput strategies as we used with the numeral recognition in Part 1 to try to minimise the overtraining, i.e. where the tes and training loss/accuracy diverge; indicating that the DNN has become too specialised on the training data and less general.

In [ ]:
input_shape = [in_size]
nn5 = models.Sequential()
reg = regularizers.l2(0.0005)

for width in [8*out_size, 2*out_size]:
    #nn5.add(layers.Dense(width, activation='relu', input_shape=input_shape))
    nn5.add(layers.Dense(width, activation='relu', input_shape=input_shape, kernel_regularizer=reg))
    nn5.add(layers.Dropout(0.2))
    input_shape = []
    
nn5.add(layers.Dense(out_size, activation='sigmoid'))
nn5.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',])

history5 = nn5.fit(train_data, train_labels, epochs=10, batch_size=512,
                   validation_data=(test_data, test_labels))

plot_training_history(history4, history5)

In [ ]:
loss, acc = nn5.evaluate(train_data, train_labels)
msg = 'Train Loss: {:.3f} Acc: {:.2f}%'.format(loss, 100.0*acc)
print(msg)
loss, acc = nn5.evaluate(test_data, test_labels)
msg = 'Test Loss: {:.3f} Acc: {:.2f}%'.format(loss, 100.0*acc)
print(msg)

Lots of tweaking to this style of network doesn't give much improvement to the test accuracy. So next we move on to consider different types of network layer.

Because we have a problem with the sparseness of amino acid sequences and generalisation to other sequences, we will introduce a layer that provides an alternative to the (one-hot) binary categorical matrix we initially used as input. This is an `Embedding` layer. In essence, this provides a lower dimensionality encoding of the amino acid sequences compared to the twenty-dimensional binary encoding. It works with the original index-encoded (0-20) form of the amino acid codes (hence `train_data_idx` etc.) and converts this to a low-dimensional vector; four in this case.

A general encoding of the amino acids into a single, simple vector form doesn't work especially well. However, the Embedding layer will adopt a different encoding for each sequence position, which are learned (i.e. optimised) during the DNN training. The new layer is added before the Dense layers and its output must be flattened into a vector.

In [ ]:
win_size = 60
pred_width = 5
train, test, in_size, out_size, prot_data = prepare_data('set160.labels', win_size, # or set243.labels
                                              pred_width, to_cat=False)
(train_data_idx, train_labels) = train
(test_data_idx, test_labels) = test

In [ ]:
reg = regularizers.l2(0.0001)

nn6 = models.Sequential()
nn6.add(layers.Embedding(NAA, 4, input_length=in_size))
nn6.add(layers.Flatten()) # Make a 1D vector

for width in [out_size*8, out_size*2]:
    nn6.add(layers.Dense(width, activation='relu', kernel_regularizer=reg))
    nn6.add(layers.Dropout(0.2))
    #nn6.add(layers.BatchNormalization())

nn6.add(layers.Dense(out_size, activation='sigmoid'))
nn6.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',])

history6 = nn6.fit(train_data_idx, train_labels, epochs=10, batch_size=512,
                   validation_data=(test_data_idx, test_labels))

As we can see by looking at the training history, this DNN performs better, especially with regards to the loss, and has even less over-trainaing. This can be attributed to the Embedding layer's encoding; likely the 4D positional encoding of the different amino acids represents axes of general predictive characteristics that extends better to unseen sequence combinations. An analogous, though general, predictive characteristic is residue hydrophobicity.

In [ ]:
plot_training_history(history5, history6)

We can also see how this improvement changes the graph of test sequence scores. The indide and outside categories are not so distinct, but the scores along the sequence are perhaps smoother:

In [ ]:
plot_seq_predict(nn6, seq1, win_size, in_size, pred_width, categorical=False)
plot_seq_predict(nn6, seq2, win_size, in_size, pred_width, categorical=False)

## <font color=purple>Exercise 3: Adding more data</font>

<font color=purple>Compare the above network (`nn6`) when run on the two different input data files 'set160.labels' and 'set243.labels' Visualise both training histories.
</font>

In [ ]:
# Exercise code

## Examining errors

As should be common practice with training DNNs, we will investigate out data to see where the worst errors occur, to see if that gives us any clue about how to improve the training. Alternatively, we may find a problem with our training or test data, which might have mistakes... 

To help with this we first define a function `plot_seq_known` that simply plots the known categorical labels for a protein as a line graph that we can directly compare with the predictions, in a similar way to `plot_seq_predict()`.

In [ ]:
# Get worst input sub_seq
def plot_seq_known(known, title):
  opts = {'alpha':0.4, 'linewidth':3}
  i, o, m, x = known.T

  fig, ax = plt.subplots()
  fig.set_size_inches(16.0, 3.0)  

  ax.set_title(title)
  ax.plot(i, color='#B0B040', label='Inside', **opts)
  ax.plot(o, color='#4080FF', label='Outside', **opts)
  ax.plot(m, color='#B00000', label='Membrane', **opts)
  ax.plot(x, color='#808080', label='Edge', linestyle='--', **opts)
  ax.set_xlabel('Sequence position')
  ax.legend()
  plt.show()

Next we loop through the protein data that we have bee storing in `prot_data`, which is output when we loaded the data files. This contains a dictionary, keyed by protein name, to record the proteins, sequence, known categorical labels and whether it was in the main training set (or test set).

We take proteins that are in the test set and compare the predicted scores from a DNN with the known labels (converted to a binary categorical encoding). The comparison involves simply calculating the distance between the output categorcal and known vector, then the iverall score for the protein is the average of this. Finally the proteins with the accompanying socres are sorted worst-first.

In [ ]:
test_nn = nn6
seq_scores = []

for prot_name in sorted(prot_data):
    seq, labels, is_main = prot_data[prot_name]
    
    if not is_main:
        scores = get_seq_scores(test_nn, seq, win_size, in_size, pred_width, categorical=False)
        known = to_categorical(np.array([LABEL_IDX[l] for l in labels]), num_classes=len(CLASSES))
        d2 = (scores - known)**2
        dist = d2.sum(axis=1) ** 0.5
        seq_score = dist.mean()
        seq_scores.append((seq_score, prot_name, seq, known))

seq_scores.sort(reverse=True)

After collating the scores we can then inspect the test protein's prediction score and known categorical profiles (i.e. along the sequence) by using out helper functions. 

In [ ]:
idx = 1
plot_seq_predict(test_nn, seqs[idx], win_size, in_size, pred_width, categorical=False)
plot_seq_known(knowns[idx], prot_names[idx])

Lastly, we augment our DNN with another new type of layer: a `Conv1D`. As the name suggests, this is not a fully-connected layer. This is a convolutional layer; these feature heavily in the next course on images. The idea here is that the network tries to learn a number of fixed-width patterns (often called 'filters'). These patterns are short vectors that are applied to only a small patch of the input. Effectively this creates a local vocabulary of sub-structures within the training input. The notion is that the range of patterns is limited and so they become general, and match better on unseen data.

In [ ]:
n_patterns = 11
w_pattern = 11
reg = regularizers.l2(0.0001)

nn7 = models.Sequential()
nn7.add(layers.Embedding(NAA, 4, input_length=in_size))

nn7.add(layers.Conv1D(n_patterns, w_pattern, activation='relu',))
nn7.add(layers.Flatten())

for width in [out_size*8, out_size*2]:
    nn7.add(layers.Dense(width, activation='relu', kernel_regularizer=reg))
    nn7.add(layers.Dropout(0.2))

nn7.add(layers.Dense(out_size, activation='sigmoid'))
nn7.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',])

In [ ]:
history7 = nn7.fit(train_data_idx, train_labels, epochs=20, batch_size=512,
                   validation_data=(test_data_idx, test_labels))

The training of this DNN takes more time than the others, but some improvement is visible.

In [ ]:
plot_training_history(history6, history7)

Pleasingly, the predictions are generally smoother for adjascent residues.

In [ ]:
plot_seq_predict(nn7, seq1, win_size, in_size, pred_width, categorical=False)
plot_seq_predict(nn7, seq2, win_size, in_size, pred_width, categorical=False)

## <font color=purple>Exercise 4: Getting above 95%</font>

<font color=purple>Try to tweak the above DNN to get a test accuracy of >95% within 20 iterations. You can adjust the window sizes, output prediction with and internal DNN parameters.
</font>

In [ ]:
Exercise code

win_size = # Fill in here
pred_width = #  Fill in here

train, test, in_size, out_size, prot_data = prepare_data('set243.labels', win_size,
                                              pred_width, to_cat=False)
(train_data_idx, train_labels) = train
(test_data_idx, test_labels) = test

n_patterns = 11 # Maybe adjust 
w_pattern = 11 # Maybe adjust 
reg = regularizers.l2(0.0001) # Maybe adjust 

nn8 = models.Sequential()
nn8.add(layers.Embedding(NAA, 4, input_length=in_size)) # Maybe adjust from 4D

nn8.add(layers.Conv1D(n_patterns, w_pattern, activation='relu',))
nn8.add(layers.Flatten())

for width in [out_size*8, out_size*2]: # Maybe adjust 
    nn8.add(layers.Dense(width, activation='relu', kernel_regularizer=reg))
    nn8.add(layers.Dropout(0.2))

nn8.add(layers.Dense(out_size, activation='sigmoid'))
nn8.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',])

plot_training_history(history7, history8)

## Going further...

That's the end of this course. However, there are a few ways that we might like to take this prediction in the future. Firstly, we can make much better use of fully aqueous proteins (i.e. no TM span), which could be used for fully 'inside' or fully 'ouside' sequence examples and serve as negative training examples. Secondly, we could expand the number of protein sequenxces by using close homologues, though here we would have to carefully balance the sizes of the protein families, and make sure they dont spread across the test/train split. Thirdly, we could predict leading signal peptides (e.g. for entry into the secretory system) and any cleavage site at the same time, which would hopfully remove a source of false positive errors.